In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from imutils import perspective
from imutils import contours
import imutils
import os

In [2]:
disparityMapScale = 35
imageScale = 25
disparityMap = []
isTrain = False
average = 0
sensorHeight = 33.3 #hard to find, so calculated beforehand using known height, known distance, and pixel.
cameraFocalLength = 28
distance = 0
points = []
boardShape = (7,7)
calibrator = []

In [3]:
testLeftImage = cv2.imread('images/test/image1/left.jpg')
testRightImage = cv2.imread('images/test/image1/right.jpg')
image = []

In [4]:
def calibrate():
    if os.path.isfile('./camera/mtxs.npy') and os.path.isfile('./camera/dists.npy'):
        return np.load('./camera/mtxs.npy'), np.load('./camera/dists.npy')
    else:
        imgpoints = []
        objpoints = []
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        objp = np.zeros((1, boardShape[0] * boardShape[1], 3), np.float32)
        objp[0,:,:2] = np.mgrid[0:boardShape[0], 0:boardShape[1]].T.reshape(-1, 2)
        for filename in os.listdir("images/calibrator2"):
            if (filename.find(".jpg") != -1):

                img = cv2.imread(os.path.join("images/calibrator2",filename))
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                ret, corners = cv2.findChessboardCorners(gray, boardShape, 
                                                         cv2.CALIB_CB_ADAPTIVE_THRESH + 
                                                         cv2.CALIB_CB_FAST_CHECK + 
                                                         cv2.CALIB_CB_NORMALIZE_IMAGE)
                if ret == True:
                    print(filename)
                    objpoints.append(objp)
                    corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
                    imgpoints.append(corners)
#                     cv2.drawChessboardCorners(img, (7,6), corners2, ret)
#                     cv2.imshow('img', resize(img, 25))
#                     cv2.waitKey(500)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        saveCameraCalibrator(mtx, dist)
        
        return mtx, dist

In [5]:
def resize(img, percentage):
    return cv2.resize(img, (int(img.shape[1] * percentage / 100), 
                                int(img.shape[0] * percentage / 100)),
                               interpolation = cv2.INTER_AREA)

In [6]:
def downSample(image, times = 1):
    for i in range(0,times):
        image = cv2.pyrDown(image)
    return image

In [7]:
def getCameraMatrix(image):
    h,  w = image.shape[:2]
    newCameraMtx, roi = cv2.getOptimalNewCameraMatrix(calibrator[0], calibrator[1], (w,h), 1, (w,h))
    return newCameraMtx, roi

In [8]:
def getDistance(event, x, y, flags, param):
    global disparityMap, isTrain, average, distance
#     EVENT_LBUTTONDOWN
#     EVENT_LBUTTONDBLCLK
    if event == cv2.EVENT_LBUTTONDOWN:
        total = 0
        resizedDisparityMap = resize(disparityMap, disparityMapScale)
        for u in range(-1,2):
            for v in range(-1,2):
                total += resizedDisparityMap[y + u, x + v]
        average = total / 9
        
        if isTrain:
            print(average)
        else:
#             = -6E-13x4 + 6E-09x3 - 1E-05x2 - 0.0392x + 163.54
#             = -2E-09x3 + 2E-05x2 - 0.1015x + 199.39
            distance = -0.000000002*average**(3) + 0.00002*average**(2) - 0.1015*average + 199.39
#             distance = -0.0000000000006*average**(4) + 0.000000006*average**(3) - 0.00001*average**(2) - 0.0392*average + 163.54
            distance = np.around(distance*10,decimals=2)
            print(distance)
#             return average

In [9]:
def saveCameraCalibrator(mtx, dist):
    np.save("./camera/mtxs", mtx)
    np.save("./camera/dists", dist)

In [10]:
def getDisparity(imageLeft, imageRight):
    cameraMtx = getCameraMatrix(imageLeft)
    undistortedL = cv2.undistort(imageLeft, calibrator[0], calibrator[1], None, cameraMtx[0])
    undistortedR = cv2.undistort(imageRight, calibrator[0], calibrator[1], None, cameraMtx[0])

    undistortedL = downSample(undistortedL, 1)
    undistortedR = downSample(undistortedR, 1)
    
    winSize = 5 #3
    minDis = -10
    maxDis = 326
    numDis = maxDis - minDis
    
    stereo = cv2.StereoSGBM_create(
        minDisparity= minDis,
        numDisparities = numDis,
        blockSize = winSize,
        uniquenessRatio = 3,
        speckleWindowSize = 100,
        speckleRange = 32,
        disp12MaxDiff = 5,
        P1 = 8*3*winSize**2,
        P2 =32*3*winSize**2
    )

    return stereo.compute(undistortedL, undistortedR)

In [11]:
def train():
    global disparityMap
    folderName =  "images/train"
    cv2.namedWindow("image") 
    cv2.setMouseCallback("image", getDistance)
    distanceToAverageRatio = []
    for folder in os.listdir(folderName):
        if (folder.isdigit()):
            distFolder = os.path.join(folderName, folder)
            print(distFolder)
            print(os.listdir(distFolder))
            imgL = cv2.imread(os.path.join(distFolder, 'left.jpg'))
            imgR = cv2.imread(os.path.join(distFolder, 'right.jpg'))
            disparityMap = getDisparity(imgL, imgR)
            show = cv2.normalize(disparityMap, dst = None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            while True:
                cv2.imshow("image", resize(show, disparityMapScale)) 
                key = cv2.waitKey(1) & 0xFF
                
                # press 'q' to continue 
                if key == ord("q"):
                    print(average)
                    distanceToAverageRatio.append([folder, average])
                    break
    cv2.destroyAllWindows()
    return distanceToAverageRatio

In [12]:
def writeToFile(distanceToAverageRatio):
    with open('distanceToAverageRatio4.txt', 'w') as filehandle:
        for ratio in distanceToAverageRatio:
            for val in ratio:
                filehandle.write('%s '%val)
            filehandle.write('\n')

In [13]:
def drawAreaToMeasure(event, x, y, flags, param): 
    global points, image
  
    if event == cv2.EVENT_LBUTTONDOWN: 
        points = [(x, y)] 
  
    elif event == cv2.EVENT_LBUTTONUP: 
        points.append((x, y)) 
        cv2.rectangle(image, points[0], points[1], (0, 255, 0), 2)
        dimension = calculateDimension()
        cv2.putText(image, "Height: {:.1f} cm".format(dimension[0]), (50,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
        cv2.putText(image, "Width: {:.1f} cm".format(dimension[1]), (50,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
        cv2.imshow("image", image)

In [14]:
def selectAreaToMeasure():
    global image
    image = testLeftImage.copy()
    image = resize(image, imageScale)
    clone = image.copy()
    cv2.namedWindow("image") 
    cv2.setMouseCallback("image", drawAreaToMeasure) 

    while True: 
        cv2.imshow("image", image) 
        key = cv2.waitKey(1) & 0xFF

        # press 'r' to reset
        if key == ord("r"): 
            image = clone.copy() 

        # press 'q' to continue 
        elif key == ord("q"): 
            break

    cv2.destroyAllWindows()  

In [15]:
def calculateDistance():
    global disparityMap, testLeftImage, testRightImage
    cv2.namedWindow("image") 
    cv2.setMouseCallback("image", getDistance)
    
    disparityMap = getDisparity(testLeftImage, testRightImage)
    show = cv2.normalize(disparityMap, dst = None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    while True:
        cv2.imshow("image", resize(show, disparityMapScale)) 
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
    cv2.destroyAllWindows()

In [16]:
def calculateDimension():
    global distance, points
    #since it was resized, we need to get the real pixel inthe original image
    heightInPixel = (points[1][1]/(imageScale/100)) - (points[0][1]/(imageScale/100))
    print(testLeftImage.shape[0])
    realHeight = (distance * sensorHeight * heightInPixel) / (cameraFocalLength * testLeftImage.shape[0])
    realWidth = (realHeight * testLeftImage.shape[1])/testLeftImage.shape[0]
    return [np.round(realHeight*0.1, 2), np.round(realWidth*0.1, 2)]  #so it is in cm

In [17]:
def run():
    global calibrator
    calibrator = calibrate()
    if isTrain:
        distanceToAverageRatio = train()
        writeToFile(distanceToAverageRatio)
    else:
        calculateDistance()
        selectAreaToMeasure()

In [18]:
run()

759.77
759.77
4032
